In [3]:
import pandas as pd
import json

In [4]:
df = pd.read_csv("/data/raw_stock_indicator.csv")

In [5]:
def json_parser(x):
    try:
        return json.loads(x)["symbol"]
    except Exception as e:
        print(e)
        return ""

In [6]:
df["company_symbol"] = df["info_json"].map(lambda x: json_parser(x))  

In [7]:
df["company_symbol"].unique()

array(['LTIM.NS', '20MICRONS.NS', '21STCENMGM.NS', '3MINDIA.NS',
       '3PLAND.NS', '5PAISA.NS', '63MOONS.NS', 'A2ZINFRA.NS', 'AAKASH.NS',
       'AARON.NS', 'AARTIDRUGS.NS', 'AARTIIND.NS', 'AARVEEDEN.NS',
       'AARVI.NS', 'AAVAS.NS', 'ABAN.NS', 'ABB.NS', 'POWERINDIA.NS',
       'ACC.NS', 'ACCELYA.NS', 'ACCURACY.NS', 'ACEINTEG.NS', 'ACE.NS',
       'ADANIENT.NS', 'ADANIGREEN.NS', 'ADANIPORTS.NS', 'ADANIPOWER.NS',
       'ADANITRANS.NS', 'ADFFOODS.NS', 'ABCAPITAL.NS', 'ABFRL.NS',
       'BIRLAMONEY.NS', 'ADORWELD.NS', 'ADROITINFO.NS', 'ADVENZYMES.NS',
       'ADVANIHOTR.NS', 'AEGISCHEM.NS', 'AFFLE.NS', 'AGARIND.NS',
       'AGRITECH.NS', 'AGROPHOS.NS', 'ATFL.NS', 'AHLADA.NS',
       'AHLUCONT.NS'], dtype=object)

In [11]:
df.columns

Index(['Tax Effect Of Unusual Items', 'Tax Rate For Calcs',
       'Normalized EBITDA', 'Total Unusual Items',
       'Total Unusual Items Excluding Goodwill',
       'Net Income From Continuing Operation Net Minority Interest',
       'Reconciled Depreciation', 'Reconciled Cost Of Revenue', 'EBIT',
       'Net Interest Income', 'Interest Expense', 'Normalized Income',
       'Net Income From Continuing And Discontinued Operation',
       'Total Expenses', 'Diluted Average Shares', 'Basic Average Shares',
       'Diluted EPS', 'Basic EPS', 'Diluted NI Availto Com Stockholders',
       'Net Income Common Stockholders', 'Otherunder Preferred Stock Dividend',
       'Net Income', 'Minority Interests',
       'Net Income Including Noncontrolling Interests',
       'Net Income Continuous Operations', 'Tax Provision', 'Pretax Income',
       'Other Non Operating Income Expenses', 'Special Income Charges',
       'Restructuring And Mergern Acquisition',
       'Net Non Operating Interest Inco

In [21]:
cols = ["Total Revenue", "Operating Revenue", "Gross Profit", "Year", "Month", "company_symbol" ,"company_symbol", "Open", "Volume"]

In [16]:
df1 = df[cols]

,Total Revenue,Operating Revenue,Gross Profit,Year,Month,company_symbol,company_symbol,Open,Volume
0,8.702100e+10,8.702100e+10,3.103400e+10,2023,6,LTIM.NS,LTIM.NS,5084.918182,581634.0
1,8.691000e+10,8.691000e+10,3.110300e+10,2023,3,LTIM.NS,LTIM.NS,4603.711157,328337.0
2,8.620000e+10,8.620000e+10,3.147800e+10,2022,12,LTIM.NS,LTIM.NS,NaN,NaN
3,4.836700e+10,4.836700e+10,1.751800e+10,2022,9,LTIM.NS,LTIM.NS,4345.868216,410491.0
4,4.522800e+10,4.522800e+10,1.650500e+10,2022,6,LTIM.NS,LTIM.NS,4019.437045,374541.0
...,...,...,...,...,...,...,...,...,...
199,7.636123e+09,7.636123e+09,1.609054e+09,2023,6,AHLUCONT.NS,AHLUCONT.NS,613.000000,37669.0
200,8.630525e+09,8.630525e+09,1.800475e+09,2023,3,AHLUCONT.NS,AHLUCONT.NS,555.000000,129257.0
201,7.432501e+09,7.432501e+09,1.351223e+09,2022,12,AHLUCONT.NS,AHLUCONT.NS,NaN,NaN
202,6.228436e+09,6.228436e+09,1.179904e+09,2022,9,AHLUCONT.NS,AHLUCONT.NS,421.100006,13806.0


In [20]:
df1["time_lag"] = df1.Year.str.cat(df1.Month.str)

AttributeError: Can only use .str accessor with string values!